In [1]:
import cv2
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import utils as utils

In [2]:
def seg_threshold(image):
        
    image = image.astype(np.uint8)
    shifted = cv2.pyrMeanShiftFiltering(image, 10,10)
    del image
    
    gray = cv2.cvtColor(shifted, cv2.COLOR_BGR2GRAY)
    del shifted
    thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    
    return thresh 
    

In [3]:
def watershed(image):
    thresh = seg_threshold(image)
    D = ndimage.distance_transform_edt(thresh)
  
    localMax = peak_local_max(D, indices=False, min_distance=20, labels=thresh)
    
    markers = ndimage.label(localMax, structure=np.ones((3, 3)))[0]
    labels = watershed(-D, markers, mask=thresh)
    
    return labels

In [4]:


def segment_image(filepath,label:np.array,heights,widths,h,w,hSize,wSize):
    img = np.zeros((hSize,wSize,3))
    raster = gdal.Open(filepath)
    for i,in1 in enumerate(heights[0:-1]):
        for j,in2 in enumerate(widths[0:-1]):

            img = utils.raster2array(raster,3,in2,in1,wSize,hSize).astype(np.uint8)


            img1 = seg_threshold(img)         


            label[in1:(in1+hSize), in2:(in2+wSize)] = img1

        print(i+1, '/',(len(heights)-1), end="\r")
    raster = None       
    print("")    
    print('image thresholded')

def adjust_height(filepath,label,heights,w,wSize,hSize):
    raster = gdal.Open(filepath)
    for i,in1 in enumerate(heights[0:-1]):
        
        img = utils.raster2array(raster,3,w-wSize,in1,wSize,hSize).astype(np.uint8)
        
        img1 = seg_threshold(img)
        del img

        label[in1:(in1+hSize), w-wSize:w] = img1
        del img1      
        print(i+1, '/',(len(heights)-1), end="\r")
    raster = None       
    print("")    
    print('height adjusted')


def adjust_width(filepath,label,widths,h,wSize,hSize):
    raster = gdal.Open(filepath)
    for i,in2 in enumerate(widths[0:-1]):
        
        img = utils.raster2array(raster,3,in2,h-hSize,wSize,hSize).astype(np.uint8)
        
        img1 = seg_threshold(img)
        del img

        label[h-hSize:h, in2:(in2+wSize)] = img1
        del img1      
        print(i+1, '/',(len(widths)-1), end="\r")
    raster = None       
    print("")    
    print('width adjusted')  

def segment_full_image(filepath):
    
    raster = gdal.Open(filepath)
    w,h = round(raster.RasterXSize), round(raster.RasterYSize)
    raster = None
    hSize = 256
    wSize = 256

    heights = list(range(0,h+1,hSize))
    widths = list(range(0,w+1,wSize))

    lHeight = len(heights)
    lWidth = len(widths)

    label = np.zeros((h,w))
    
    segment_image(filepath,label,heights,widths,h,w,hSize,wSize)
    adjust_height(filepath,label,heights,w,wSize,hSize)
    adjust_width(filepath,label,widths,h,wSize,hSize)
    raster = gdal.Open(filepath)
    img = utils.raster2array(raster,3,w-wSize,h-hSize,wSize,hSize).astype(np.uint8)
    label[h-hSize:,w-wSize:] = seg_threshold(img)
    raster = None
    return label

In [5]:

image_folder = './images'
label_folder = './labels'

size = 256
#List of all jps and tif files
filepaths = ((utils.get_all_images(image_folder, 'jpg')))

for i,filepath in enumerate(filepaths):
    label = segment_full_image(filepath)
    cv2.imwrite("./{}/cltn{}.tif".format(label_folder,i+1), label.astype(np.uint8))  

    print('label saved for file: {} / {}'.format(i+1,len(filepaths)))      



1 / 1
image thresholded
1 / 1
height adjusted
1 / 1
width adjusted
label saved for file: 1 / 54
1 / 1
image thresholded
1 / 1
height adjusted
1 / 1
width adjusted
label saved for file: 2 / 54
1 / 1
image thresholded
1 / 1
height adjusted
1 / 1
width adjusted
label saved for file: 3 / 54
1 / 1
image thresholded
1 / 1
height adjusted
1 / 1
width adjusted
label saved for file: 4 / 54
1 / 1
image thresholded
1 / 1
height adjusted
1 / 1
width adjusted
label saved for file: 5 / 54
1 / 1
image thresholded
1 / 1
height adjusted
1 / 1
width adjusted
label saved for file: 6 / 54
1 / 1
image thresholded
1 / 1
height adjusted
1 / 1
width adjusted
label saved for file: 7 / 54
1 / 1
image thresholded
1 / 1
height adjusted
1 / 1
width adjusted
label saved for file: 8 / 54
1 / 1
image thresholded
1 / 1
height adjusted
1 / 1
width adjusted
label saved for file: 9 / 54
1 / 1
image thresholded
1 / 1
height adjusted
1 / 1
width adjusted
label saved for file: 10 / 54
1 / 1
image thresholded
1 / 1
height ad